[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/DeZero-Koki/blob/master/from_scratch_3/steps/step17.ipynb)

# 제1고지 미분 자동 계산

In [1]:
import numpy as np


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [2]:
# class Exp(Function):
#     def forward(self, x):
#         return np.exp(x)

#     def backward(self, gy):
#         x = self.input.data
#         gx = np.exp(x) * gy
#         return gx


# def exp(x):
#     return Exp()(x)


In [3]:
def numerical_diff(f, x, eps=1e-4):
    """

      param:
        f (Function): 미분의 대상이 되는 함수
        x (Variable): 미분을 계산하는 변수
        eps (float): 작은 값
    """
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 제2고지 자연스러운 코드로

## 11단계 가변 길이 인수(순전파 편)

## 12단계 가변 길이 인수(개선 편)

## 13단계 가변 길이 인수(역전파 편)

## 14단계 같은 변수 반복 사용

## 15단계 복잡한 계산 그래프(이론 편)

## 16단계 복잡한 계산 그래프(구현 편)

In [4]:
# class Add(Function):
#     def forward(self, x0, x1):
#         y = x0 + x1
#         return y

#     def backward(self, gy):
#         return gy, gy


# def add(x0, x1):
#     return Add()(x0, x1)


## 17단계 메모리 관리와 순환 참조

### 17.1 메모리 관리

### 17.2 참조 카운트 방식의 메모리 관리

In [5]:
# class obj: pass
# def f(x): print(x)

# a = obj()  # 변수에 대입: 참조 카운트 1
# f(a)  # 함수에 전달: 함수 안에서는 참조 카운트 2
# # 함수 완료: 빠져나오면 참조 카운트 1
# a = None  # 대입 해제: 참조 카운트 0

In [6]:
# a = obj()
# b = obj()
# c = obj()

# a.b = b
# b.c = c

# a = b = c = None  # a의 참조 카운트가 0이 되면서 삭제되고, b와 c도 따라서 삭제 됨

### 17.3 순환 참조

In [7]:
# a = obj()
# b = obj()
# c = obj()

# a.b = b
# b.c = c
# c.a = a

# a = b = c = None  # a, b, c의 참조 카운트 중 어느 것도 0이 되지 않아 접근할 수 없으면서도 메모리에서 삭제되지 않는 상태가 됨

### 17.4 weakref 모듈

- 여기 내용은 jupyter로 실행하면 제대로 실행되지 않음

In [8]:
python_script = """
import weakref
import numpy as np

a = np.array([1, 2, 3])
b = weakref.ref(a)

print(b)
print(b())
""".strip()

with open("py.py", 'w') as f: f.write(python_script)
!python py.py

<weakref at 0x78cbb55f11c0; to 'numpy.ndarray' at 0x78cbb54a5b90>
[1 2 3]


In [9]:
python_script = """
import weakref
import numpy as np

a = np.array([1, 2, 3])
b = weakref.ref(a)

a = None
print(b)
# print(b())
""".strip()

with open("py.py", 'w') as f: f.write(python_script)
!python py.py

<weakref at 0x7c398b96d1c0; dead>


In [10]:
import weakref


class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, *inputs):
        xs = [ x.data for x in inputs ]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple): ys = ys,
        outputs = [ Variable(as_array(y)) for y in ys ]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]  # 변경!
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [11]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [ output().grad for output in f.outputs ]  # 변경!
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple): gxs = gxs,

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

    def cleargrad(self):
        self.grad = None


In [12]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


In [13]:
for i in range(10):
    x = Variable(np.random.randn(10000))  # 거대한 데이터
    y = square(square(square(x)))  # 복잡한 계산을 수행한다.